In [29]:
import keras
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
import numpy as np

In [30]:
batch_size = 128
num_classes = 49

# input image dimensions
img_rows, img_cols = 28, 28

def load(f):
    return np.load(f)['arr_0']

# Load the data
x_train = load('k49-train-imgs.npz')
x_test = load('k49-test-imgs.npz')
y_train = load('k49-train-labels.npz')
y_test = load('k49-test-labels.npz')

In [31]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('{} train samples, {} test samples'.format(len(x_train), len(x_test)))

232365 train samples, 38547 test samples


In [32]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [33]:
class DynamicAugmentationCallback(Callback):
    def __init__(self, x_train, y_train, batch_size):
        self.x_train = x_train
        self.y_train = y_train
        self.batch_size = batch_size

        # Inicializar un generador de datos básico
        self.datagen = ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            fill_mode='nearest'
        )
        self.datagen.fit(self.x_train)

    def on_epoch_begin(self, epoch, logs=None):
        # Ajustar los parámetros del generador dinámicamente en función de la época
        rotation_range = 10 + (epoch % 20)  # Rotación dinámica
        zoom_range = 0.1 + (epoch % 10) / 100
        self.datagen = ImageDataGenerator(
            rotation_range=rotation_range,
            zoom_range=zoom_range,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.1,
            fill_mode='nearest'
        )
        self.datagen.fit(self.x_train)

    def on_epoch_end(self, epoch, logs=None):
        print(f"Época {epoch + 1}: Data augmentation actualizado con rotation_range={10 + (epoch % 20)}")

    def get_flow(self):
        return self.datagen.flow(self.x_train, self.y_train, batch_size=self.batch_size)


In [34]:
# Cargar el modelo previamente entrenado
model = load_model('kanji_model_350_epochs.hdf5')

# Compilar el modelo con el nuevo optimizador
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

In [36]:
epochs = 350

# Crear el callback para augmentación dinámica
dynamic_augmentation = DynamicAugmentationCallback(x_train, y_train, batch_size)

# Entrenar el modelo
for epoch in range(epochs):
    print(f"\nEntrenando época {epoch + 1}/{epochs}")
    generator = dynamic_augmentation.get_flow()
    model.fit(
        generator,
        steps_per_epoch=len(x_train) // batch_size,
        epochs=1,  # Entrenar una época a la vez
        validation_data=(x_test, y_test),
        verbose=1
    )



Entrenando época 1/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 206s 114ms/step - accuracy: 0.6413 - loss: 1.3457 - val_accuracy: 0.7643 - val_loss: 0.9011

Entrenando época 2/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 223s 123ms/step - accuracy: 0.6405 - loss: 1.3435 - val_accuracy: 0.7648 - val_loss: 0.9014

Entrenando época 3/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 202s 111ms/step - accuracy: 0.6411 - loss: 1.3439 - val_accuracy: 0.7647 - val_loss: 0.9005

Entrenando época 4/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 226s 124ms/step - accuracy: 0.6438 - loss: 1.3358 - val_accuracy: 0.7648 - val_loss: 0.8996

Entrenando época 5/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 216s 119ms/step - accuracy: 0.6432 - loss: 1.3407 - val_accuracy: 0.7655 - val_loss: 0.8999

Entrenando época 6/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 222s 122ms/step - accuracy: 0.6424 - loss: 1.3347 - val_accuracy: 0.7652 - val_loss: 0.8994

Entrenando época 7/350
1815/1815 ━━━━━━━━━━━━━━━━━━━━ 212s 117ms/step - accuracy: 0.6422 - loss: 1.3375 - val_accurac

In [37]:
model.save('kanji_model_with_dynamic_augmentation.hdf5')

In [38]:
# Evaluación del modelo actualizado
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

Test loss: 0.7888981699943542
Test accuracy: 0.7933691143989563
